In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment

from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram
import yaml
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = [18, 5]

In [ ]:
config_file = "config.yaml"
with open(config_file, "r") as f:
    config = yaml.safe_load(f.read())

In [ ]:
audio = AudioSegment.from_mp3("/data/whale/20230920_002737Z.mp3")
audio

In [ ]:
class MarsClip(object):

    def __init__(self, filepath):
        self.filepath = filepath
        self.directory, self.filename = os.path.split(self.filepath)
        self.audio = AudioSegment.from_mp3(filepath)
        self.samples = np.array(self.audio.get_array_of_samples())[::2] # the samples are interleaved channel samples, channels are equal (mono)
        

    def view_spectrogram(self):
        '''View spectrogram using matplotlib specgram function
        '''
        sxx, freqs, t, _ = plt.specgram(self.samples, NFFT=512, noverlap=0, cmap='jet')
        plt.show()


    def get_spec_img(self):
        '''Calculate spectrogram using scipy.signal.spectrogram
        Returns:
            sxx: numpy array of spectrogram data
        '''
        f, t, sxx = spectrogram(self.samples, **config['spectrogram_params'])
        return sxx, f, t
    
    def plot_spec_img(self):
        sxx, f, t = self.get_spec_img()
        # plt.imshow(np.flipud(10*np.log10(sxx)), cmap='jet')
        def forward(x):
            return abs(x)**.8

        def inverse(x):
            return abs(x)**(1/.8)

        plt.yscale("function", functions=(forward, inverse))
        plt.ylim((0, config['spectrogram_params']['fs']//2))       
        plt.pcolormesh(t, f, 10*np.log10(sxx), cmap='jet')
        plt.show()
        return f
    
    def get_filename(self):
        print(self.filename)
        return self.filename
    
    def play(self):
        return self.audio
    
    def mv_whale(self):
        new_path = os.path.join("/data/whale/", self.filename)
        os.rename(self.filepath, new_path)
        self.filepath = new_path
        return self.filepath
        

    def mv_nowhale(self):
        new_path = os.path.join("/data/no_whale/", self.filename)
        os.rename(self.filepath, new_path)
        self.filepath = new_path
        return self.filepath

    def cp_interesiting(self):
        new_path = os.path.join("/data/interesting/", self.filename)
        os.rename(self.filepath, new_path)
        self.filepath = new_path
        return self.filepath

In [ ]:
class DataDir(object):

    def __init__(self, directory):
        self.directory = directory
        self.file_list = os.listdir(directory)
    
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        filepath = self.file_list[self.index]
        self.index += 1
        return self.directory + filepath


In [ ]:
data = DataDir("/data/data/")
datafile_iter = iter(data)

In [ ]:
my_clip = MarsClip(next(datafile_iter))
my_clip.plot_spec_img()
my_clip.get_filename()
my_clip.play()

In [ ]:
my_clip.mv_whale()

In [ ]:
my_clip.mv_nowhale()

In [ ]:
ax = np.array(audio.get_array_of_samples())[::2] # the samples are interleaved channel samples, channels are equal (mono)
f, t, sxx = spectrogram(x, **config['spectrogram_params'])
print(sxx.shape)
plt.imshow(np.flipud(10*np.log10(sxx)), cmap='jet')
plt.show()

In [ ]:
N = 512
fs = audio.frame_rate
T = 1.0/fs
xs = fftfreq(N, T)[:N//2]
ys = 10*np.log10(np.abs(fft(x, n=N)))[:N//2]
# plt.plot(xs, ys)
# plt.show()

In [ ]:
x = x[:N*(x.size//N)]
x = x.reshape((x.size//N, N))

In [ ]:
ys = []
for segment in x:
    ys.append(10*np.log10(np.abs(fft(segment)))[:N//2])
ys = np.array(ys)

In [ ]:
plt.imshow(np.flipud(ys.T), cmap='viridis')
plt.show()